# Performance analysis

In [ ]:
%matplotlib inline
import re
from datetime import datetime
import numpy as np
from mpmath import mp
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import math
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
plt.rcParams['figure.dpi'] = 90
plt.rcParams['figure.figsize'] = [24.0, 16.0]
plt.rcParams['text.latex.unicode'] = False
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'stixsans'
plt.rcParams['font.family'] = 'DejaVu Sans'

## Log parsing

In [ ]:
log = open("../../pedersen-large.log","r").read()

In [ ]:
start = None
events = []
for line in log.split('\n'):
    match = re.search(r'^\[(.+Z) TRACE [:\w]+\] (BEGIN|END) (.*)$', line)
    if match:
        (time, event, name) = match.groups()
        time = datetime.strptime(time, '%Y-%m-%dT%H:%M:%S.%fZ')
        if start:
            time -= start
            time = time.total_seconds()
        else:
            start = time
            time = 0
        begin = event == 'BEGIN'
        events += [(time, begin, name)]

In [ ]:
stack = []
spans = []
histogram = {}
last = None
for event in events:
    (time, begin, name) = event
    # Collect top of stack histogram
    if last:
        top_of_stack = stack[-1][2]
        duration = time - last
        if top_of_stack in histogram:
            histogram[top_of_stack] += duration
        else:
            histogram[top_of_stack] = duration
    last = time
    # Compute spans
    if begin:
        stack += [event]
    else:
        (start, _, previous_name) = stack[-1]
        stack = stack[:-1]
        assert name == previous_name
        spans += [(len(stack), start, time, name)]
histogram = {k: v for k, v in sorted(histogram.items(), key=lambda item: -item[1])}
print(len(spans), len(histogram))

## Flame graph

In [ ]:
labels = list(set([span[3] for span in spans]))
colour_map = mpl.cm.rainbow(np.linspace(0, 1, len(labels)))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim((0,140))
ax.set_ylim((0,7))

for span in spans:
    (depth, start, end, label) = span
    colour = colour_map[labels.index(label)]
    rectangle = mpl.patches.Rectangle((start, depth), end - start, 1, fc=colour, ec='white')
    ax.add_patch(rectangle)
    if end - start > 3:
        ax.text(start+0.5, depth+0.05, label)

## Histogram

In [ ]:
labels = list(histogram.keys())
values = np.array(list(histogram.values()))
y_pos = np.arange(len(labels))

fig, ax = plt.subplots()
ax.set_yticks(y_pos)
ax.set_yticklabels(labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Total time $sec$')
ax.barh(y_pos, values)